In [5]:
!pip install duckdb pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import duckdb
import pandas as pd

con = duckdb.connect("dev_database.duckdb")
con.sql("USE dev_database.dev_sode")
con.sql("SHOW TABLES;").show()


┌────────────────┐
│      name      │
│    varchar     │
├────────────────┤
│ ghcn_2010      │
│ ghcn_2011      │
│ ghcn_2012      │
│ ghcn_2013      │
│ ghcn_2014      │
│ ghcn_2015      │
│ ghcn_2016      │
│ ghcn_2017      │
│ ghcn_2018      │
│ ghcn_2019      │
│     ·          │
│     ·          │
│     ·          │
│ noaa_ghcn_2013 │
│ noaa_ghcn_2014 │
│ noaa_ghcn_2015 │
│ noaa_ghcn_2016 │
│ noaa_ghcn_2017 │
│ noaa_ghcn_2018 │
│ noaa_ghcn_2019 │
│ noaa_ghcn_2020 │
│ noaa_ghcn_2021 │
│ noaa_ghcn_2022 │
├────────────────┤
│    26 rows     │
│   (20 shown)   │
└────────────────┘



In [4]:
df = con.sql("select * from fact_daily_climate_parameters ").df()

In [7]:
con.close()

## Stations

In [13]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-stations.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('stations.txt','w') as f:
    f.write(text_body)




NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [7]:
df_stations = pd.read_fwf('stations.txt')
df_stations.columns=['station_id','latitude','longitude','state','station_name','gsn_flag','hcn_crn_flag','wmo_number']
df_stations.to_csv('stations.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 3964: invalid continuation byte

## Countries

In [8]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-countries.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('countries.txt','w') as f:
    f.write(text_body)




In [9]:
df_countries = pd.read_fwf('countries.txt')
df_countries.columns=['country_code','country_name']
df_countries.to_csv('countries.csv')

# States

In [10]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-states.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('state.txt','w') as f:
    f.write(text_body)

df_states = pd.read_fwf('state.txt')
df_states.columns=['state_code','state_name']
df_states.to_csv('states.csv')

In [11]:
# df_stations= df.join(df_countries,on=)
def push_metadata(data, database_name, table_name):

    conn = duckdb.connect(database=database_name,read_only=False)
    conn.execute("CREATE SCHEMA IF NOT EXISTS dev_sode;")
    df = pd.read_csv(data)

    conn.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

    conn.close()

In [12]:
push_metadata('states.csv', "dev_database.duckdb", "states")
push_metadata('stations.csv',"dev_database.duckdb", "stations")
push_metadata('countries.csv', "dev_database.duckdb", "countries")

FileNotFoundError: [Errno 2] No such file or directory: 'stations.csv'